In [ ]:
!python --version

Python 3.7.15


In [ ]:
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia.smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
!pip install tensorflow==1.14
!pip install keras==2.2.5
!pip install mrcnn
!pip install h5py==2.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109.3 MB 31 kB/s 
     |████████████████████████████████| 3.1 MB 38.3 MB/s 
     |████████████████████████████████| 488 kB 7.9 MB/s 
     |████████████████████████████████| 50 kB 3.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.2
    Uninstalling tensorflow-2.9.2:
      Successfully uninstalled tensorflow-2.9.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is

In [ ]:
# import psutil
# import humanize
# import GPUtil as GPU

import os
import sys
import json
import datetime
import numpy as np
import skimage.draw

from mrcnn.config import Config
from mrcnn import model as modellib, utils

from google.colab import drive

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
GPUs = GPU.getGPUs()
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free " + humanize.naturalsize(psutil.virtual_memory().available), " | Proc size: " + humanize.naturalsize(process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free 10.8 GB  | Proc size: 2.8 GB
GPU RAM Free: 15109MB | Used: 0MB | Util   0% | Total 15109


In [ ]:
drive.mount('/content/drive/')
ROOT_DIR = '/content/drive/MyDrive/pythoncode/insuranceApp'

Mounted at /content/drive/


In [ ]:
sys.path.append(ROOT_DIR)

COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

class CustomConfig(Config):

    NAME = "object"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 2
    STEPS_PER_EPOCH = 50
    DETECTION_MIN_CONFIDENCE = 0.9

class CustomDataset(utils.Dataset):
    def load_custom(self, dataset_dir, subset):

        self.add_class("object", 1, "scratch")
        self.add_class("object", 2, "dent")
        
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir + subset)

        annotations1 = json.load(open(os.path.join(dataset_dir + '/' + "via_region_data_json.json"),'r',encoding="utf8",errors='ignore'))
        annotations = list(annotations1.values())

        annotations = [a for a in annotations if a['regions']]
        
        for a in annotations:
            
            polygons = [r['shape_attributes'] for r in a['regions']]
            objects = [s['region_attributes']['name'] for s in a['regions']]
            print("objects: ",objects)
            name_dict = {"scratch": 1, "dent": 2}

            num_ids = [name_dict[a] for a in objects]

            print("numids", num_ids)

            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",
                image_id=a['filename'],
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids)
            
    def load_mask(self, image_id):
        
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids

    def image_reference(self, image_id):

        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

def train(model):

    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom(ROOT_DIR + '/dataset/', "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom(ROOT_DIR + '/dataset/', "val")
    dataset_val.prepare()

    print("Training network heads")
    model.train(dataset_train,dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=15,
                layers='heads')

In [ ]:
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH

if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

train(model)







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


... done downloading pretrained model!
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent', 'dent']
numids [2, 2]
objects:  ['dent']
numids [2]
objects:  ['dent', 'dent']
numids [2, 2]
objects:  ['dent', 'dent']
numids [2, 2]
objects:  ['dent', 'scratch', 'scratch', 'scratch', 'scratch']
numids [2, 1, 1, 1, 1]
objects:  ['dent', 'scratch']
numids [2, 1]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent', 'scratch', 'scratch']
numids [2, 1, 1]
objects:  ['dent']
numids [2]
objects:  ['dent', 'scratch']
numids [2, 1]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent', 'scratch']
numids [2, 1]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent', 'scratch']
numids [2, 1]
objects:


/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple worke

Epoch 1/15


/usr/local/lib/python3.7/dist-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/usr/local/lib/python3.7/dist-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/usr/local/lib/python3.7/dist-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. 

...

/usr/local/lib/python3.7/dist-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)


27/50 [===============>..............] - ETA: 22:26 - loss: 1.8698 - rpn_class_loss: 0.0147 - rpn_bbox_loss: 0.8334 - mrcnn_class_loss: 0.1360 - mrcnn_bbox_loss: 0.5008 - mrcnn_mask_loss: 0.3848

/usr/local/lib/python3.7/dist-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)


28/50 [===============>..............] - ETA: 21:27 - loss: 1.8685 - rpn_class_loss: 0.0146 - rpn_bbox_loss: 0.8189 - mrcnn_class_loss: 0.1367 - mrcnn_bbox_loss: 0.4972 - mrcnn_mask_loss: 0.4012

/usr/local/lib/python3.7/dist-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
